## Candidate evaluation with metrics @100

In this notebook I considered weeks 102-104 and I computed the metrices for articles and customer style in that period. 
1. Then I found the 100 articles that are more similar to the customer stile preferences and I used them as candidates.

#### Import datasets and libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
articles = pd.read_parquet("generated_data/articles_with_fabric_namenum.parquet")
articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
0,108775015,108775,12855,253,9,0,1010016,0,9,0,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
2,108775051,108775,44846,253,9,0,1010017,3,11,11,...,0,1,0,16,30,1002,2,8834,[0],[jersey]
3,110065001,110065,8159,306,13,4,1010016,0,9,0,...,7,1,0,61,5,1017,4,8243,[1],[microfibre]
4,110065002,110065,8159,306,13,4,1010016,0,10,2,...,7,1,0,61,5,1017,4,8243,[1],[microfibre]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,24043,302,14,7,1010014,6,9,0,...,2,3,3,26,16,1021,13,28026,"[4, 13, 3]","[cotton, knit, elastic]"
105538,953763001,953763,24027,253,9,0,1010016,0,9,0,...,0,1,0,2,15,1005,0,28025,[8],[polyester]
105539,956217002,956217,24040,265,1,2,1010016,0,9,0,...,0,1,0,18,12,1005,0,28024,[0],[jersey]
105540,957375001,957375,24041,72,48,3,1010016,0,9,0,...,1,2,2,52,25,1019,1,28023,"[5, 6]","[plastic, hair]"


In [3]:
a2 = pd.read_csv("data/articles.csv")
a2

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [4]:
style_cols = ["graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]
for col in style_cols:
    print(col)
    print(a2[col].unique())
    print()

graphical_appearance_name
['Solid' 'Stripe' 'All over pattern' 'Melange' 'Transparent' 'Metallic'
 'Application/3D' 'Denim' 'Colour blocking' 'Dot' 'Other structure'
 'Contrast' 'Treatment' 'Check' 'Chambray' 'Front print'
 'Glittering/Metallic' 'Mixed solid/pattern' 'Placement print'
 'Other pattern' 'Neps' 'Embroidery' 'Lace' 'Jacquard' 'Unknown' 'Argyle'
 'Slub' 'Mesh' 'Sequin' 'Hologram']

perceived_colour_master_name
['Black' 'White' 'Beige' 'Grey' 'Blue' 'Pink' 'Lilac Purple' 'Red' 'Mole'
 'Orange' 'Metal' 'Brown' 'Turquoise' 'Yellow' 'Khaki green' 'Green'
 'undefined' 'Unknown' 'Yellowish Green' 'Bluish Green']

section_name
['Womens Everyday Basics' 'Womens Lingerie'
 'Womens Nightwear, Socks & Tigh' 'Baby Essentials & Complements'
 'Men Underwear' 'Mama' 'Womens Small accessories' 'Men H&M Sport'
 'Kids Boy' 'Divided Basics' 'Girls Underwear & Basics' 'Mens Outerwear'
 'Womens Big accessories' 'Divided Accessories'
 'Womens Swimwear, beachwear' 'Divided Selected' 'Boys Underwe

In [5]:
customers = pd.read_parquet("data/customers.parquet")
customers

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,6883939031699146327,-1,-1,0,0,49,6305
1,11246327431398957306,-1,-1,0,0,25,33726
2,18439897732908966680,-1,-1,0,0,24,3247
3,18352672461570950206,-1,-1,0,0,54,168643
4,18162778555210377306,1,1,0,1,52,168645
...,...,...,...,...,...,...,...
1371975,7551062398649767985,-1,-1,0,0,24,50351
1371976,9305341941720086711,-1,-1,0,0,21,80169
1371977,10160427316885688932,1,1,0,1,21,106737
1371978,2551401172826382186,1,1,0,1,18,111894


In [6]:
c2 = pd.read_csv("data/customers.csv")
c2

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
...,...,...,...,...,...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,NaN,NaN,ACTIVE,NONE,24.0,7aa399f7e669990daba2d92c577b52237380662f36480b...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,NaN,NaN,ACTIVE,NONE,21.0,3f47f1279beb72215f4de557d950e0bfa73789d24acb5e...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,1.0,1.0,ACTIVE,Regularly,21.0,4563fc79215672cd6a863f2b4bf56b8f898f2d96ed590e...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.0,1.0,ACTIVE,Regularly,18.0,8892c18e9bc3dca6aa4000cb8094fc4b51ee8db2ed14d7...


In [7]:
c_to_c = customers[["customer_id"]]
c_to_c.loc[:, "cust_original"] = c2["customer_id"]
c_to_c

C:\Users\lucre\AppData\Local\Temp\ipykernel_13292\2078248447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_to_c.loc[:, "cust_original"] = c2["customer_id"]


,customer_id,cust_original
0,6883939031699146327,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,11246327431398957306,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,18439897732908966680,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,18352672461570950206,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,18162778555210377306,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...
...,...,...
1371975,7551062398649767985,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...
1371976,9305341941720086711,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...
1371977,10160427316885688932,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...
1371978,2551401172826382186,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...


In [8]:
transactions = pd.read_parquet("data/transactions_train.parquet")
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
25784,2018-09-20,1728846800780188,519773001,0.028458,2,0
25785,2018-09-20,1728846800780188,578472001,0.032525,2,0
5389,2018-09-20,2076973761519164,661795002,0.167797,2,0
5390,2018-09-20,2076973761519164,684080003,0.101678,2,0
47429,2018-09-20,2918879973994241,662980001,0.033881,1,0
...,...,...,...,...,...,...
31774722,2020-09-22,18439937050817258297,891591003,0.084729,2,104
31774723,2020-09-22,18439937050817258297,869706005,0.084729,2,104
31779097,2020-09-22,18440902715633436014,918894002,0.016932,1,104
31779098,2020-09-22,18440902715633436014,761269001,0.016932,1,104


In [9]:
tr = transactions.merge(articles, on="article_id")
tr

,t_dat,customer_id,article_id,price,sales_channel_id,week,product_code,prod_name,product_type_no,product_type_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,fabric_num,fabric_name
0,2018-09-20,1728846800780188,519773001,0.028458,2,0,519773,7147,245,17,...,0,1,0,15,0,1003,3,10231,"[13, 24]","[knit, wool]"
1,2018-09-20,12679047700498299353,519773001,0.035576,2,0,519773,7147,245,17,...,0,1,0,15,0,1003,3,10231,"[13, 24]","[knit, wool]"
2,2018-09-20,17920410454051211922,519773001,0.035576,2,0,519773,7147,245,17,...,0,1,0,15,0,1003,3,10231,"[13, 24]","[knit, wool]"
3,2018-09-21,12044778652893604504,519773001,0.035576,2,0,519773,7147,245,17,...,0,1,0,15,0,1003,3,10231,"[13, 24]","[knit, wool]"
4,2018-09-21,12044778652893604504,519773001,0.035576,2,0,519773,7147,245,17,...,0,1,0,15,0,1003,3,10231,"[13, 24]","[knit, wool]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31788319,2020-09-22,17026193828065643786,533261032,0.033881,2,104,533261,280,256,29,...,5,4,1,44,2,1002,2,185,"[4, 0]","[cotton, jersey]"
31788320,2020-09-22,17104273831947945561,875646001,0.033881,2,104,875646,3070,272,0,...,3,4,1,46,8,1009,5,31807,"[4, 3, 22]","[cotton, elastic, twill]"
31788321,2020-09-22,17843582181630128591,757903026,0.015237,1,104,757903,357,80,28,...,6,1,0,65,29,1019,1,166,[-1],None
31788322,2020-09-22,17970366705591601158,892893001,0.135576,2,104,892893,29006,252,2,...,0,1,0,14,36,1003,3,27251,"[13, 45, 24]","[knit, cashmere, wool]"


In [10]:
tr_this_fall = tr[(tr.week > 101) & (tr.week <= 104)]
print(tr_this_fall.shape)

(760209, 32)


In [11]:
tr_this_fall.to_parquet("generated_data/trans_art_102_to_104.parquet")

In [12]:
cus_this_fall = tr_this_fall["customer_id"].unique().tolist()
len(cus_this_fall)

182321

In [13]:
art_this_fall = tr_this_fall["article_id"].unique().tolist()
len(art_this_fall)

25899

In [14]:
purch = tr_this_fall.groupby("customer_id")["article_id"]

### Articles Matrix

In [15]:
index_to_art_id = pd.DataFrame({'article_id': art_this_fall})

In [16]:
index_to_art_id.sort_values(by='article_id')
index_to_art_id['original_index'] = index_to_art_id.index
index_to_art_id.head()

,article_id,original_index
0,578472001,0
1,613246004,1
2,688873001,2
3,673911002,3
4,568601006,4


In [17]:
articles_this_fall = articles[articles['article_id'].isin(art_this_fall)]

In [18]:
articles_this_fall_fabric = articles_this_fall[['article_id', 'fabric_num']]
articles_this_fall_style = articles_this_fall[["article_id", "graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]]

print(articles_this_fall_fabric.shape)
print(articles_this_fall_style.shape)

(25899, 2)
(25899, 5)


In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

articles_this_fall_fabric = articles_this_fall_fabric.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(articles_this_fall_fabric.pop('fabric_num')),
                index=articles_this_fall_fabric.index,
                columns=mlb.classes_))

articles_this_fall_fabric.head()

,article_id,-1,0,1,2,3,4,5,6,7,...,91,92,94,95,97,99,100,101,103,104
1,108775044,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,111565001,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,111586001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,111593001,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,111609001,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
articles_this_fall_fabric = articles_this_fall_fabric.drop(['article_id'], axis=1)

In [21]:
one_hot_encoded = pd.get_dummies(articles_this_fall[['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name']], columns=['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name'])
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,graphical_appearance_name_0,graphical_appearance_name_1,graphical_appearance_name_2,graphical_appearance_name_3,graphical_appearance_name_4,graphical_appearance_name_5,graphical_appearance_name_6,graphical_appearance_name_7,graphical_appearance_name_8,graphical_appearance_name_9,...,section_name_49,section_name_50,section_name_52,section_name_54,section_name_55,index_group_name_0,index_group_name_1,index_group_name_2,index_group_name_3,index_group_name_4
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105536,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
105537,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
105538,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
merged_matrix_art = one_hot_encoded.join(articles_this_fall_fabric)

In [23]:
merged_matrix_art = csr_matrix(merged_matrix_art.values)
print(merged_matrix_art)

  (0, 0)	1
  (0, 31)	1
  (0, 79)	1
  (0, 103)	1
  (0, 109)	1
  (1, 0)	1
  (1, 29)	1
  (1, 80)	1
  (1, 103)	1
  (1, 111)	1
  (2, 0)	1
  (2, 29)	1
  (2, 80)	1
  (2, 103)	1
  (2, 108)	1
  (3, 0)	1
  (3, 29)	1
  (3, 80)	1
  (3, 103)	1
  (3, 112)	1
  (4, 0)	1
  (4, 29)	1
  (4, 80)	1
  (4, 103)	1
  (4, 108)	1
  :	:
  (25894, 103)	1
  (25894, 112)	1
  (25894, 122)	1
  (25895, 1)	1
  (25895, 43)	1
  (25895, 61)	1
  (25895, 103)	1
  (25895, 109)	1
  (25896, 6)	1
  (25896, 29)	1
  (25896, 65)	1
  (25896, 106)	1
  (25896, 112)	1
  (25896, 113)	1
  (25896, 122)	1
  (25897, 0)	1
  (25897, 29)	1
  (25897, 64)	1
  (25897, 103)	1
  (25897, 117)	1
  (25898, 0)	1
  (25898, 29)	1
  (25898, 61)	1
  (25898, 103)	1
  (25898, 109)	1


In [24]:
print(merged_matrix_art.shape)

(25899, 200)


### Customer style matrix

In [25]:
index_to_cus_id = pd.DataFrame({'customer_id': cus_this_fall})

In [26]:
index_to_cus_id.sort_values(by='customer_id')
index_to_cus_id['original_index'] = index_to_cus_id.index
index_to_cus_id.head()

,customer_id,original_index
0,5316172648860712200,0
1,6474552940938576885,1
2,5701871825794564345,2
3,6949189610971841595,3
4,2430626937109945466,4


In [27]:
customers_this_fall_fabric = tr_this_fall[['customer_id', 'fabric_num']]
customers_this_fall_style = tr_this_fall[["customer_id", "graphical_appearance_name", "perceived_colour_master_name", "section_name", "index_group_name"]]

print(customers_this_fall_fabric.shape)
print(customers_this_fall_style.shape)

(760209, 2)
(760209, 5)


In [28]:
print(index_to_cus_id.shape)

(182321, 2)


In [29]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

customers_this_fall_fabric = customers_this_fall_fabric.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(customers_this_fall_fabric.pop('fabric_num')),
                index=customers_this_fall_fabric.index,
                columns=mlb.classes_))

customers_this_fall_fabric.head()

,customer_id,-1,0,1,2,3,4,5,6,7,...,91,92,94,95,97,99,100,101,103,104
252,5316172648860712200,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,6474552940938576885,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6203,5701871825794564345,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10396,6949189610971841595,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10704,2430626937109945466,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
customers_this_fall_fabric = customers_this_fall_fabric.groupby('customer_id').sum()
customers_this_fall_fabric.head()

KeyboardInterrupt: 

In [ ]:
customers_this_fall_fabric.sort_values(by='customer_id')

In [ ]:
sparse_cus_fabric = csr_matrix(customers_this_fall_fabric.values)
sparse_cus_fabric

In [ ]:
one_hot_encoded = pd.get_dummies(tr_this_fall[['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name']], columns=['graphical_appearance_name', 'perceived_colour_master_name', 'section_name', 'index_group_name'])
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

In [ ]:
one_hot_encoded['customer_id'] = tr_this_fall['customer_id']
one_hot_encoded.head()

In [ ]:
one_hot_encoded = one_hot_encoded.groupby('customer_id').sum()
print(one_hot_encoded.shape)
one_hot_encoded.head()

In [ ]:
merged_cus = pd.merge(customers_this_fall_fabric, one_hot_encoded, on="customer_id", how="inner")
merged_cus.sort_values(by='customer_id')
merged_cus

In [ ]:
merged_cus = merged_cus.reset_index()
merged_cus = merged_cus.drop("customer_id", axis=1)
print(merged_cus.shape)
merged_cus.head()

In [ ]:
merged_cus.columns = merged_cus.columns.astype(str)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
merged_cus = scaler.fit_transform(merged_cus)

In [ ]:
matrice_sparsa_2 = csr_matrix(merged_cus)
print(matrice_sparsa_2)

In [ ]:
similarity_matrix_sparse = cosine_similarity(matrice_sparsa_2, merged_matrix_art, dense_output=False)

In [ ]:
import util
%run util.py

In [ ]:
toprank = get_top_K_ranks(similarity_matrix_sparse, 100)
print(toprank)

In [ ]:
V = toprank.data  # Valori non zero
U = toprank.indices  # Indici delle righe corrispondenti ai valori non zero
I = toprank.indptr  # Indici delle colonne corrispondenti ai valori non zero

# Ad esempio, puoi stampare i primi 5 valori, indici delle righe e indici delle colonne
print("Valori (V):", V[:100])
print("Indici delle righe (U):", U[:100])
print("Indici delle colonne (I):", I[:100])

In [ ]:

num_elements = len(V)

neg_train = pd.DataFrame(columns=['id', 'index'])

# Per ogni primo elemento, estrai gli indici corrispondenti
for i in range(len(I) - 1):
    start_idx = I[i]
    end_idx = I[i + 1]
    
    # Gli indici corrispondenti al primo elemento
    first_element_indices = U[start_idx:end_idx]
    
    df_to_append = pd.DataFrame({'id': [i], 'index': [first_element_indices]})
    
    neg_train = pd.concat([neg_train, df_to_append], ignore_index=True)
    print(f"Primo elemento {i}: {first_element_indices}")

In [ ]:
input_string = neg_train["index"][0]

In [ ]:
import re

numbers = re.findall(r'\d+', input_string)

# Converting the numbers to integers
numbers = [int(num) for num in numbers]

# Displaying the result
print(numbers)

In [ ]:
neg_train["customer_id"] = index_to_cus_id["customer_id"]
neg_train.head()

In [ ]:
import re

def map_ids_to_article_id(index_list):
    
    numbers = re.findall(r'\d+', index_list)
    # Converting the numbers to integers
    numbers = [int(num) for num in numbers]
    
    return [index_to_art_id.loc[index_to_art_id['original_index'] == i, 'article_id'].values[0] for i in numbers]


# Applying the mapping function to the 'idex' column in neg_train
neg_train['article_ids'] = neg_train['index'].apply(map_ids_to_article_id)

neg_train

In [ ]:
neg_train.columns

In [ ]:
neg_train = neg_train.drop(['id', 'index'], axis=1)
neg_train.head()

In [48]:
neg_train.to_parquet("generated_data/first_100_candidates.parquet", index=False)

### Metrics@K

In [30]:
prop = pd.read_parquet("generated_data/first_100_candidates.parquet")
prop

,customer_id,article_ids
0,5316172648860712200,"[574109011, 306847009, 665303001, 570002002, 1..."
1,6474552940938576885,"[615939001, 657014002, 478646002, 537738001, 5..."
2,5701871825794564345,"[615939001, 657014002, 478646002, 537738001, 4..."
3,6949189610971841595,"[548111002, 582258001, 490176001, 373506004, 6..."
4,2430626937109945466,"[592773001, 301227002, 715138001, 763490002, 7..."
...,...,...
182316,14854476225233177976,"[636323005, 611503005, 606800001, 717490007, 7..."
182317,1035453850503293971,"[300024013, 189616001, 615939001, 657014002, 4..."
182318,7173948812548054370,"[417951001, 622314008, 665506003, 594453003, 7..."
182319,7208500580267335569,"[548111002, 582258001, 490176001, 373506004, 6..."


In [31]:
type(prop.article_ids[0][0])

numpy.int64

In [32]:
import ast

In [33]:
def recall_at_k(actual, predicted, k):
    # actual: list of actual relevant items
    # predicted: list of predicted items
    # k: top k recommendations

    # Get the top k predicted items
    predicted_at_k = predicted[:k]

    # Calculate the number of common items between actual and predicted
    common_items = set(actual) & set(predicted_at_k)

    # Calculate Recall at k
    recall_at_k = len(common_items) / len(actual) if len(actual) > 0 else 0
    return recall_at_k

In [34]:
def precision_at_k(actual, predicted, k):
    # actual: list of actual relevant items
    # predicted: list of predicted items
    # k: top k recommendations

    # Get the top k predicted items
    predicted_at_k = predicted[:k]

    # Calculate the number of common items between actual and predicted
    common_items = set(actual) & set(predicted_at_k)

    # Calculate Precision at k
    precision_at_k = len(common_items) / k if k > 0 else 0
    return precision_at_k

In [35]:
def average_precision_recall_at_k(actual, predicted, k):
    # actual_dict: dictionary where keys are user IDs and values are lists of actual relevant items
    # predicted_dict: dictionary where keys are user IDs and values are lists of predicted items
    # k: top k recommendations

    total_precision = 0.0
    total_recall = 0.0

    # Iterate over users
    for customer_id, group_data in purch:
        #print(customer_id)
        actual_items = purch.get_group(customer_id).tolist()
        #print(len(actual_items))
        find_line = prop[prop['customer_id'] == customer_id]
        predicted_it = find_line['article_ids'].tolist()
        predicted_items = predicted_it[0]
        #predicted_items = ast.literal_eval(predicted_it)
        #print(len(predicted_items))

        # Calculate precision and recall at k for each user
        precision = precision_at_k(actual_items, predicted_items, k)
        recall = recall_at_k(actual_items, predicted_items, k)

        # Accumulate precision and recall for averaging
        total_precision += precision
        total_recall += recall

    # Calculate average precision and recall
    avg_precision = total_precision / len(actual)
    avg_recall = total_recall / len(actual)

    return avg_precision, avg_recall

In [36]:
k = 100

avg_precision, avg_recall = average_precision_recall_at_k(purch, prop, k)

print(f'Average Precision at {k}: {avg_precision}')
print(f'Average Recall at {k}: {avg_recall}')

Average Precision at 100: 0.00014128926453892403
Average Recall at 100: 0.003567908227134897
